In [39]:
import tensorflow as tf

tf.reset_default_graph()
graph = tf.get_default_graph()

X = tf.placeholder(tf.float32)
a = tf.Variable(tf.truncated_normal([1]))
b = tf.Variable(tf.truncated_normal([1]))
loss = tf.losses.mean_squared_error(a, b)
learn = tf.train.AdamOptimizer().minimize(loss)
init = tf.global_variables_initializer()

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./board', graph)

sess = tf.Session()
sess.run(init)

summary = sess.run(merged, feed_dict={X: 1})




TypeError: Fetch argument None has invalid type <class 'NoneType'>

In [ ]:
writer.add_summary(summary, 0)

In [17]:
import tensorflow as tf

tf.reset_default_graph()
graph = tf.get_default_graph()

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

add = tf.add(X, Y)

writer = tf.summary.FileWriter('./board', sess.graph)
writer.add_graph(graph)

In [86]:
import pandas as pd
import time

from gensim.models import Word2Vec
import tensorflow as tf


def get_sess():
    tf.reset_default_graph()
    graph = tf.get_default_graph()
    lt = tf.constant(
        pd.read_pickle('./data/lookup_table'),
        dtype=tf.float32
    )
    sen = tf.truncated_normal([4, 64, 512])
    batch = tf.map_fn(lambda word: tf.map_fn(
        lambda x: tf.cast(tf.argmin(
            tf.norm((lt-x), axis=1)
        ), dtype=tf.float32), word
    ), sen)
    sess = tf.Session()
    return batch, sess, graph


def batch2sens(batch):
    
    i2w = Word2Vec.load('./data/w2v_model').wv.index2word
    
    def get_sen(l):
        ws = ["<BLANK>" if i == len(i2w) else i2w[i] for i in l]
        sen = ' '.join(w.split('-')[0] for w in ws)
        return sen

    batch = batch.astype(int)
    sens = [get_sen(l) for l in batch]
    for sen in sens:
        print(sen)


batch, sess, graph = get_sess()
batch_ = sess.run(batch)
batch2sens(batch_)

뱔 비평계 현원 다이엘 사묘 에미 월야 마셜 튀니지 산인 잡류 리베리 창동 배런 깍지 바히나지오칸테 한명호 내취 만드셔 법무법인 식수대 미겔 엘더스 안소영 홍희관 아세이 아세이 신주쿠교엔 현대산업개발 혐녀 문무왕 나무판자 좌완 북해도 읕것 가루다 하빌랜드 김경민 뛰어다님 디선 됴앙 보도비첸코브 땁시다 절러 시종 김수임 강원 마고리 이열 반만년 첟 금지옥엽 괜차늠 똭똭 왈왈왈 쫑쫑 산세바스티안 감성돔 혐녀 꾸뻑 저기 김승호 비요른 장자크
부르디외 쓰러져버렸 사진학 카밀 옴니아 줜나 장하 신춘문예 당했을까 빠구리 배리무어 쳉 엄영 몽사 껄떡 미복 게스리 허종호 강경론 미뉴엣 윤수일 시펑 김두봉 이혜리 부르디외 벌게져 히로타 항저우 괴로워진다 탤론 아마미 봣엇어요 뺴기냐 오발 전대 진이가 콘찰로프스키 콘도 바뀐다는데 벌 제께 법무법인 동호대교 양민영 퇴계 방성대곡 잔학성 태류 안재훈 마스다 똘똘한 도언 돕슨 주먹구구 배선 키시다 불가형언 산세바스티안 국봉 천연가스 괜츈한듯 살로 장하 각지
김홍 츈 오동통 디브이 애재라 몽사 서태 이뤄지긴 디파이 씡난다아 급경사 월야 뽀옹 이뤄지긴 움킬 크레치만 벌라 서태 리버럴 입이 획일주의 획일주의 서상구 광장 원기 쿄오 간복 로렐라이 컴비네이션 백창우 쁑쁑류 정찰대 재은 능교 내당 걸 게스리 비바체 김두환 반음정 호우 안옥 팰런 윤무 쁑쁑류 누런색 뱅크헤드 승방 카약 하나무라 헣헣 끅 정채봉 제호 몽사 감김 무한리필 개기름 다포 백건 큽니당 접어 정진 고상지
라펠 난바 전동현 보도비첸코브 저격술 락앤락 콕 차야 마카취일 총관 띳이즈쓰빠르따 정몽 인조반정 산세바스티안 합니데이 저격술 장록 원당 갸냘프 산화 괜츈한듯 복백 김윤호 양민영 몸져 공학윤리 달라진다 알베스 견족 권미경 문사 김승호 자치 박종환 후아나 혼연 색스 능해 소격동 서태 창동 스패니시 유중원 날카로워져 쑝카 친서 장하 달일 똠양꿍 김지훈 지취 김성현 이곤 설화지 정진 띳이즈쓰빠르따 쌓인다 라펠 천신만고 품명 헛헛헛 딕켄스 한지혜 정재희
